## Predicting the future price of Apple Company using LSTM:

In [ ]:
#Importing the reqd libraries:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense,InputLayer

### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

!pip install yfinance
import yfinance as yf

In [ ]:
### Setting the Style for plots
plt.style.use('fivethirtyeight')

In [ ]:
### Getting the data from yfinance:
### AAPL is the short form for Apple.
data = yf.download("AAPL", start="2005-01-01", end="2020-01-01")

## EDA:

In [ ]:
data.head()

Clearly it represents time series data because indices are of time

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
### Checking for Missing Values:
data.isnull().sum()

In [ ]:
### Visualising the initial distribution of the data to check if it makes sense:
### Checking the Closing Price

fig,ax=plt.subplots(2,1,figsize=(16,16))

ax[0].set_title("Closing Price History")
ax[0].plot(data['Close'])
ax[0].set_xlabel("Data")
ax[0].set_ylabel("Closing Price in USD")
ax[0].legend()

### Checking the Closing Price

ax[1].set_title("Opening Price History")
ax[1].plot(data['Open'])
ax[1].set_xlabel("Data")
ax[1].set_ylabel("Opening Price in USD")
ax[1].legend()

fig.show()

In [ ]:
### Combining the above plots into one:
plt.figure(figsize=(16,7))

plt.title("Closing Price History/Opening Price History")
plt.plot(data['Close'],"b")
plt.plot(data['Open'],"y")
plt.xlabel("Data")
plt.ylabel("Closing Price in USD")
plt.legend()

plt.show()  

In [ ]:
### Zooming into one section, say dates between 2005 and 2007:

mask1 = (data.index >= '2005-01-01') & (data.index <= '2007-01-01')

fig,ax=plt.subplots(2,1,figsize=(16,16))

ax[0].set_title("Closing Price History/Opening Price History for the period 2005-2006")
ax[0].plot(data['Close'].loc[mask1],"b")
ax[0].plot(data['Open'].loc[mask1],"y")
ax[0].set_xlabel("Data")
ax[0].set_ylabel("Closing Price in USD")
ax[0].legend()

### Zooming further into one section, say dates in 2005:
mask2 = (data.index >= '2005-01-01') & (data.index <= '2006-01-01')

ax[1].set_title("Closing Price History/Opening Price History for 2005")
ax[1].plot(data['Close'].loc[mask2],"b")
ax[1].plot(data['Open'].loc[mask2],"y")
ax[1].set_xlabel("Data")
ax[1].set_ylabel("Closing Price in USD")
ax[1].legend()

fig.show()

## Data Preparation

In [ ]:
## Conducting the Forecasting on the "Close" price:
df=data["Close"]

### LSTMs are very sensitive to the scale of the data. So I apply MinMaxScaler:
scaler=MinMaxScaler(feature_range=(0,1))
df_scaled=scaler.fit_transform(np.array(df).reshape(-1,1))

In [ ]:
df_scaled

In [ ]:
##splitting dataset into train and test
n=len(df_scaled)

training_size=int(n*0.65)
test_size=n-training_size

train_data,test_data=df_scaled[0:training_size,:],df_scaled[training_size:n,:1]

In [ ]:
train_data.shape,test_data.shape

In [ ]:
### First five values of the training data
train_data[:5]

In [ ]:
train_data.reshape(len(train_data))

Now comes the **CRUX** of the data preparation step:

Step 1. Choose the number of time steps(**t**). This controls the number of data points that are going to be used to predict 
the current data point.

Step 2: Create the Dataset with t features and a target variable

**NOTE**:Time steps(t) is an important hyperparameter.

In [ ]:
def create_data(df,t):
    n=len(df)
    df=df.reshape(n)
    output=[]
    label=[]
    for x in range(n-t-1):
        output.append(list(df[x:x+t]))
        label.append(df[x+t])
    
    return np.array(output),np.array(label)

X_train,y_train=create_data(train_data,60)

X_test,y_test=create_data(test_data,60)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
X_test.shape,y_test.shape

## Model Making: (Simple LSTM)

A LSTM model is defined using the Tensorflow deep learning library. 

The model requires a three-dimensional input as:
**[samples, time steps, features]**.

Hence we reshape the data as follows:

In [ ]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
model=Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].set_title("Loss")
ax[0].plot(model.history.history['loss'])
ax[1].set_title("Test Loss")
ax[1].plot(model.history.history['val_loss'])
fig.show()

Doing the prediction and validating the predictions using performance metrics:

In [ ]:
train_predict=model.predict(X_train) 
test_predict=model.predict(X_test)

In [ ]:
### The RMSE is defined as:
### math.sqrt(mean_squared_error(actual,predicted))

### Train Error:
math.sqrt(mean_squared_error(y_train,train_predict))

### Test Error:
math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
##Transforming the data back to its original scale:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
### Train Error after back-scaling:
math.sqrt(mean_squared_error(y_train,train_predict))

### Test Error after back-scaling:
math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
### Plotting the predicted observations:

# shift train predictions for plotting
t=60
trainPredictPlot = np.empty_like(df_scaled)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[t:len(train_predict)+t, :] = train_predict

# shift test predictions for plotting
testPredictPlot = np.empty_like(df_scaled)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(t*2)+1:len(df_scaled)-1, :] = test_predict

# plot baseline and predictions
plt.figure(figsize=(10,10))
plt.title("Overlap of Original Dataset and Predictions")
plt.plot(scaler.inverse_transform(df_scaled),label="Original Data")
plt.plot(trainPredictPlot,label="Predicted Train")
plt.plot(testPredictPlot,label="Predicted Test")
plt.legend()
plt.show()

From the Above Plot, it is clearly visible that our model is has performed considerably good in retracing the original data.

## Forecasting:

For forecasting the next 30 days, we need to consider the last 60 data points present in the original test data i.e.**test_data** variable:

In [ ]:
x_input=test_data[test_data.shape[0]-60:].reshape(1,-1)
x_input.shape

temp_input=list(x_input)
temp_input=temp_input[0].tolist()
temp_input

In [ ]:
# To Forecast for next 30 Days:
lst_output=[]
n_steps=60
i=0
while(i<30):
    
    if(len(temp_input)>60): ## after first iteration, temp_input will have size of 61
        
        x_input=np.array(temp_input[1:])
       
        x_input = x_input.reshape(1,-1).reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        
        temp_input.extend(yhat[0].tolist())  ### Adding prediction to the temp_input, which will serve as test data for next iteration
        temp_input=temp_input[1:]   ### Ignores the first value, as it is not required for next price's prediction.
        
        lst_output.extend(yhat.tolist())  ### Appends the cost
        
        i=i+1
        
    else:  ### Runs initially as x_input will be of length 60 initially:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i=i+1

In [ ]:
day_new=np.arange(1,61)
day_pred=np.arange(61,91)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(df_scaled[df_scaled.shape[0]-60:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))